In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import binom, norm, skewnorm, multivariate_normal, skew
from scipy.optimize import minimize, root, Bounds
from random import sample, choices
from datetime import datetime
from bisect import bisect_left
import ot
import plotly.graph_objs as go
from plotly.offline import iplot

In [ ]:
def takeClosest(myList, myNumber):
    """
    Assumes myList is sorted. Returns closest value to myNumber.

    If two numbers are equally close, return the smallest number.
    """  
    pos = bisect_left(myList, myNumber)
    if pos == 0:
        return pos
    if pos == len(myList):
        return -1
    before = myList[pos - 1]
    after = myList[pos]
    if after - myNumber < myNumber - before:
        return pos
    else:
        return pos-1

def sum_limit_order_book(x, pointer1, pointer2, lmo_volume):
    left = pointer1[x]
    right = pointer2[x]
    if left >= right:
        return np.zeros(N)
    lmo_sum = np.sum(lmo_volume[left:right,:] , axis = 0)  
    return lmo_sum.tolist()

def imbalance_generate(x, weight, N, ask_sum, bid_sum):
    weight = np.array(weight)
    if np.sum(ask_sum[x]) == None:
        return
    B = np.sum(weight * ask_sum[x] + weight * bid_sum[x])
    A =  np.sum(weight * (bid_sum[x]))
    if B == 0:
        return
    return A / B

In [ ]:
# import the data
lob = pd.read_csv('./data/20170417_AEM_limit_order_book.csv')
lob.head()

In [ ]:
#Prepare temp column for resampling
lob['temp'] = pd.to_datetime(lob['time']).dt.tz_localize('UTC')
lob['temp_nanos'] = pd.to_timedelta(lob['time_nanos'], unit='ns')
lob['temp'] = lob['temp'] + lob['temp_nanos']
lob['time'] = pd.to_datetime(lob['time']).apply(lambda x: (x - datetime(1970,1,1)).total_seconds())
lob['time'] = lob['time'] + 1e-9  * lob['time_nanos']
lob.sort_values('time', inplace=True)

In [ ]:
#transform unit of price into ticks
name_ask = ['pa' + str(i) for i in range(10)]
name_bid = ['pb' + str(i) for i in range(10)]
lob[name_ask] = 100 * lob[name_ask]
lob[name_bid] = 100 * lob[name_bid]

In [ ]:
#Give the parameters
weight = np.array([3.55216664e-01, 1.23132520e-01, 3.67009391e-01, 1.54641425e-01,])
N = 4
nums = 28
dp_plus = np.array([2.34523512e-12, 1.45316185e-01, 3.52416071e-03, 2.51397975e-01,
        2.07630352e-01, 1.14650602e-01, 2.22077440e-01, 1.99809229e-05,
        5.53833050e-02])
mu = np.sum(dp_plus * np.arange(-N , N + 1))
q = np.array([8.69112914e-01, 5.67308008e-02, 6.61292998e-03, 1.33009922e-03,4.68992061e-04])

In [ ]:
#Compute time delta between two orders
time_ls = list(lob['time'].values)
delta_s = np.diff(time_ls).tolist()
delta_s.insert(0 ,0)
lob['time_diff'] = delta_s 

#weigh the book volume by time delta
for i in range(1, N+1):
    name_a = 'va' + str(i) 
    name_b = 'vb' + str(i) 
    name_aa = 'va-' + str(i) 
    name_bb = 'vb-' + str(i)
    lob[name_aa] = lob[name_a] * lob['time_diff']
    lob[name_bb] = lob[name_b] * lob['time_diff']
name_ask = ['va-' + str(i) for i in range(1, N+1)]
name_bid = ['vb-' + str(i) for i in range(1, N+1)]
volume_ask = np.array(lob[name_ask])
volume_bid = np.array(lob[name_bid])

Compute$\sum\limits_{t -f\leq s< t} H_s $ where $H_s$ is the market order volume at time $s$.

In [ ]:
m = lob.loc[(lob['reason'] == 'TRADE') & (lob['side'] == 'Buy') & (lob['market_state'] == 'Opening') | (lob['reason'] == 'TRADE') & (lob['side'] == 'Sell') & (lob['market_state'] == 'Open') ]
sm = - m[['book_change','temp']].resample(str(nums)+'S', on = 'temp').sum().between_time('09:30', '16:00')
H = sm['book_change'].values
sm.reset_index(drop = False, inplace = True)
T = sm.temp.values
s = np.array(list(map(lambda x: (x - datetime(1970,1,1)).total_seconds(), pd.to_datetime(T, unit = 's') )))

Compute $\hat{\imath}_-(t)$ and $\hat{\imath}_+(t)$, the imbalance before and after a market order, as follows:
\begin{align*}
    \hat{\imath}_-(t) &= \frac{\sum\limits_{k\leq N} \sum\limits_{t - f  \leq s<t} w_k \bar{v}^-_k(s)}{\sum\limits_{k\leq N} \sum\limits_{t - f  \leq s<t}  w_k \left(\bar{v}^-_k(s) + \bar{v}_k(s) \right)}\\
    \hat{\imath}_+(t) &= \frac{ \sum\limits_{k\leq N} \sum\limits_{t  \leq s<t+1} w_k \bar{v}^-_k(s)}{\sum\limits_{k\leq N} \sum\limits_{t\leq s<t+1}  w_k \left(\bar{v}^-_k(s) + \bar{v}_k(s) \right)} 
\end{align*}


In [ ]:
s1 = np.array(s) - nums
s2 = np.array(s) 
s3 = np.array(s) + 1
time_ls = lob.time.values
# find index in lob according to s
pointer1 = [takeClosest(time_ls, i) for i in s1]
pointer2 = [takeClosest(time_ls, i) for i in s2]
pointer3 = [takeClosest(time_ls, i) for i in s3]
ask_before = list(map(lambda x: sum_limit_order_book(x, pointer1, pointer2, volume_ask), range(len(pointer1))))
bid_before = list(map(lambda x: sum_limit_order_book(x, pointer1, pointer2, volume_bid), range(len(pointer1))))
ask_after = list(map(lambda x: sum_limit_order_book(x, pointer2, pointer3, volume_ask), range(len(pointer1))))
bid_after = list(map(lambda x: sum_limit_order_book(x, pointer2, pointer3, volume_bid), range(len(pointer1))))
imb_before = np.array(list(map(lambda x: imbalance_generate(x, weight, N, ask_before, bid_before), range(len(s)))))
imb_after = np.array(list(map(lambda x: imbalance_generate(x, weight, N, ask_after, bid_after), range(len(s)))))

$a_t$ is the average size of the limit order book $f$ seconds before a market order
\begin{align*}
    a_t &= \frac{\sum\limits_{k = 1}^N \sum\limits_{t -f\leq s< t} \bar{v}_k(s) \Delta s}{Nf} \\
    \rho_t &= \frac{\sum\limits_{k = 1}^N \sum\limits_{t -f\leq s< t} H_s }{a_t}
\end{align*} 
where $H_s$ is the market order volume at time $s$.

In [ ]:
A = np.array(list(map(lambda x: np.sum(ask_before[x]), range(len(ask_before))))) 
R = N * H / A
idd = np.where(~np.isnan(R))
R = R[idd]
l1 = imb_before[idd]
l2 = imb_after[idd]
s = s[idd]
idd = np.where( (R>0) )[0]
R = R[idd]
l1 = l1[idd]
l2 = l2[idd]
s = s[idd]
idd = np.where(l2!= np.array(None))
R = R[idd]
l1 = l1[idd]
l2 = l2[idd]
s = s[idd]

For each $\hat{\imath}_-(t)$, the optimal spoofing strategy $v_{spoof}$ can be solved explicitly from
\begin{equation*}
    v_{spoof,k} = 1+\frac{ \left(1-\hat{\imath}_-(t) \right) w_k}{Q}\left[2\rho_t w_k \mu^+ \frac{1-\hat{\imath}_-(t)}{\hat{\imath}_-(t)} \imath^2 - \left( Q_k \rho_t +\nu \right)\right]^+ 
\end{equation*}
where $\imath = \frac{b_t}{b_t + a_t + w_k v_{spoof,k}}$

In [ ]:
def imath_star(imath,imbalance,rho,q,w,mu):
    Q = np.array([np.sum(q[i+1:]) for i in range(N)])
    Q_bar = np.array([np.sum(np.arange(i,N,1) * q[i+1:]) for i in range(N)])
    v = np.zeros(N)
    for place in range(N):
        v[place] = np.maximum(2 * rho * mu * weight[place] * imath ** 2 *(1-imbalance)/imbalance - (Q[place] * (rho  - place) + Q_bar[place]) ,0)  / (Q[place])
    return np.abs(imath - (imbalance)/( 1 + (1-imbalance) * np.sum(weight*v) ))

imath = []
v = np.zeros(N)
for index in range(len(l1)):
    imbalance = l1[index]
    rho = R[index]
    if rho == 0:
        i = imbalance
    else:
        i = minimize(imath_star, [imbalance], bounds = Bounds(0,imbalance), args = (imbalance, rho, q, weight, mu)).x[0]
    imath.append(i)

In [ ]:
#Parameters for bivariate skewnormal distribution
def skew_condition(y1, y2, beta, omega, Omega, alpha_0, alpha):
    xi = beta[1] + omega[0,1] * (y1 - beta[0])/ omega[0,0]
    x0 = (y1 - beta[0]) * alpha_0 / np.sqrt(omega[0,0])
    x0_prime = x0 * np.sqrt(1 + alpha[1] * Omega * alpha[1] / omega[1,1])
    return multivariate_normal.pdf(x = y2 - xi, mean = 0, cov = Omega) * norm.cdf(alpha[1] * (y2 - xi)/ np.sqrt(omega[1,1]) + x0_prime ) / norm.cdf(x0)

#Compute the wasserstein distance following by the step in the paper appendix
def wasserstein_distance(x, number, Q, l1, l2,l3):
    #Truncate the data in the window
    left = pointer1[x]
    right = pointer2[x]
    ll1 = l1[left:right + 1]
    ll2 = l2[left:right + 1]
    ll3 = l3[left:right + 1]
    q = np.quantile(ll2,Q)
    sum1 = 0
    sum2 = 0
    #Discretize the support of i_spoof_- for sampling
    X = np.linspace(0, 1, 100)
    for i in range(len(Q) - 2):
        idd = np.where((ll2 >= q[i]) & (ll2 < q[i+1]))
        a = ll1[idd]
        b = ll2[idd]
        c = ll3[idd]
        if len(b) > 0:
            #Sample number times from the marginal distribution and compute the average wasserstein diatance
            sample1 = np.array(list(map(lambda x : norm.rvs(loc = mean1 + sigma1 / sigma2 * cor * (x - mean2), scale = np.sqrt((1- cor ** 2)) * sigma1, size = number), b)))
            ssum1 = sum(np.array(list(map(lambda x : ot.wasserstein_1d(a, sample1[:,x], p = 2), range(number))))) / number
            Y = np.array([skew_condition(b, x, beta, omega, Omega, alpha_0, alpha).tolist() for x in X])
            sample2 = np.array(list(map(lambda x : choices(X, Y[:,x] / np.sum(Y[:,x]), k = number), range(len(b)))))
            ssum2 = sum(np.array(list(map(lambda x : ot.wasserstein_1d(a, sample2[:,x], p = 2), range(number))))) / number
            sum1 =  sum1 + ssum1   
            sum2 =  sum2 + ssum2 
    #Include the end point in the last group
    i = len(Q) - 2
    idd = np.where((ll2 >= q[i]) & (ll2 <= q[i+1]))
    a = ll1[idd]
    b = ll2[idd]
    c = ll3[idd]
    if len(b) > 0:
        sample1 = np.array(list(map(lambda x : norm.rvs(loc = mean1 + sigma1 / sigma2 * cor * (x - mean2), scale = np.sqrt((1 - cor ** 2)) * sigma1, size = number) , b)))
        ssum1 = sum(np.array(list(map(lambda x : ot.wasserstein_1d(a, sample1[:,x], p = 2), range(number))))) / number
        Y = np.array([skew_condition(b, x, beta, omega, Omega, alpha_0, alpha).tolist() for x in X])
        sample2 = np.array(list(map(lambda x : choices(X, Y[:,x] / np.sum(Y[:,x]), k = number), range(len(b)))))
        ssum2 = sum(np.array(list(map(lambda x : ot.wasserstein_1d(a, sample2[:,x], p = 2), range(number))))) / number
        sum1 = sum1 + ssum1     
        sum2 = sum2 + ssum2 
    return [sum1 / len(Q), sum2 / len(Q)]

In [ ]:
l3 = np.array(imath)
idd = np.where((l3<=1)&(l3>=0))
l1 = l1[idd]
l2 = l2[idd]
l3 = l3[idd]
s = s[idd]
R = R[idd]
idd = np.where((l2<=1)&(l2>=0))
l1 = l1[idd]
l2 = l2[idd]
l3 = l3[idd]
s = s[idd]
R = R[idd]

In [ ]:
#Save the imbalance to use R to fit the skewnormal distribution
bi_imb = np.append(l3.reshape(len(l3),1),l2.reshape(len(l3),1),axis = 1)
np.savetxt('./data/i_spoof_i_+.txt', bi_imb)

In [ ]:
#Generate the window index
window_size = 100
pointer2 = np.arange(window_size, len(l1))
pointer1 = pointer2 - window_size

In [ ]:
#The bivarite skewnrom distribution is fitted in R and here is the parameters.
beta  = np.array([0.5329954, 0.476884])
alpha = np.array([-1.196460, 1.295003 ])
omega = np.array([[0.007050057, 0.003401895], [0.003401895, 0.013884434]]) 
Omega = omega[1,1] - omega[0,1] ** 2 / omega[0,0]
alpha_0 = (alpha[0] + alpha[1] *np.sqrt(omega[1,1]) / np.sqrt(omega[0,0])) 
alpha_0 =alpha_0 / np.sqrt(1 + alpha[1] * Omega * alpha[1] / omega[1,1]) 
mean1 = np.mean(l1)
mean2 = np.mean(l2)
sigma1 = np.std(l1)
sigma2 = np.std(l2)
cor = np.corrcoef(l1,l2)[0,1] 

In [ ]:
#Divide the imbalance by quantiles
Q = np.array([0, 0.2, 0.4, 0.6, 0.8, 1])
#Number of sampling times in each window
number = 3
wd = np.array(list(map(lambda x: wasserstein_distance(x, number, Q, l1, l2, l3), range(len(pointer1)))))

In [ ]:
fig = go.Figure()
fig.add_scatter(x = np.arange(len(wd[:,0])), y= wd[:,0],)
fig.add_scatter(x = np.arange(len(wd[:,0])), y= wd[:,1],)
iplot(fig)